In [ ]:
#!pip install newsapi-python
#!pip install jugaad-data

In [2]:
# Importing libraries

from newsapi import NewsApiClient
import pandas as pd
from datetime import date
from jugaad_data.nse import stock_df

In [3]:
# News api key 

newsapi = NewsApiClient(api_key='80f4fb54f04940d897dc1e50f981e194')

In [4]:
# Get if day was profitable or not 

def getResult(row):
    outPut = 0
    if (row['OPEN']-row['CLOSE'] > 0):
        outPut = 1
    return outPut

In [9]:
# Add result in each row

def process_row(row, df_result):
    filtered_df = df_result.loc[(df_result['date'] == row['date'])]
    if not filtered_df.empty:
        # If there are matching rows, return the first 'result' value
        return filtered_df['result'].iloc[0]
    else:
        # If no matching rows, return a default value or handle the case accordingly
        return None

In [19]:
def get_mapped_data(queryName, stockName):
    all_articles = newsapi.get_everything(q=queryName,
                                     from_param='2023-12-07',
                                      language='en')
    df_stock = stock_df(symbol=stockName, from_date=date(2023,12,2),
            to_date=date(2024,1,7))
    df_stock["result"] = df_stock.apply(lambda row: getResult(row), axis=1)
    df_result = df_stock[['DATE', 'result']].copy()
    df_result = df_result.rename(columns={'DATE': 'date'})
    df_result['date'] = pd.to_datetime(df_result.date, utc=True).dt.date
    df_article = pd.json_normalize(all_articles['articles'])
    df_article = df_article.rename(columns={'publishedAt': 'date'})
    df_article['date'] = pd.to_datetime(df_article.date, utc=True).dt.date
    df_article['result'] = df_article.apply(process_row, axis=1, df_result=df_result)
    return df_article
    

In [20]:
df_article = get_mapped_data('HDFC Bank Ltd OR HDFC','HDFCBANK' )

In [21]:
df_article

,author,title,description,url,urlToImage,date,content,source.id,source.name,result
0,Ashutosh R Shyam,"After a lacklustre 2023, will 2024 be a year o...","Mega stocks such as HDFC Bank, Reliance Indust...",https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-106428582,wid...",2023-12-31,"SynopsisMega stocks such as HDFC Bank, Relianc...",the-times-of-india,The Times of India,NaN
1,Michail,"iQOO 12 launched in India, starting at INR 52,999",The iQOO 12 is now official in India serving a...,https://www.gsmarena.com/iqoo_12_launched_in_i...,https://fdn.gsmarena.com/imgroot/news/23/12/iq...,2023-12-12,The iQOO 12 is now official in India serving a...,None,GSMArena.com,1.0
2,ETMarkets.com,"F&O stocks to buy or sell today: ICICI Bank, M...",The S&P BSE Sensex rose nearly 400 points whil...,https://economictimes.indiatimes.com/markets/o...,"https://img.etimg.com/thumb/msid-106199210,wid...",2023-12-22,The Indian market is likely to trade higher on...,the-times-of-india,The Times of India,1.0
3,Tech Desk,iPhone 15 at Rs 35K? Check out these lucrative...,iPhone 15 is among the most sought after smart...,https://indianexpress.com/article/technology/i...,https://images.indianexpress.com/2023/12/iPhon...,2023-12-29,The year is almost ending. Other than thumping...,None,The Indian Express,0.0
4,PTI,Stock markets climb for 2nd day on buying in I...,"Among the Sensex firms, Wipro jumped over 6 pe...",https://indianexpress.com/article/business/mar...,https://images.indianexpress.com/2023/12/BSE-1...,2023-12-22,Stock markets extended gains for a second day ...,None,The Indian Express,1.0
...,...,...,...,...,...,...,...,...,...,...
95,ET Now,"Expect some more cuts in small, midcap segment...",“ if you look at most of the largecap names li...,https://economictimes.indiatimes.com/markets/e...,"https://img.etimg.com/thumb/msid-106172881,wid...",2023-12-21,"Amnish Aggarwal, Head-Reasearch, Prabhudas Lil...",the-times-of-india,The Times of India,0.0
96,Moneycontrol News,"Shriram Finance could replace UPL, attracting ...",The official list for the Nifty 50 rejig will ...,https://www.moneycontrol.com/news/business/mar...,https://images.moneycontrol.com/static-mcnews/...,2023-12-12,Nuvama's strategists predict Shriram Finance L...,None,Moneycontrol,1.0
97,Moneycontrol News,Outlook 2024: Corporate chieftains highly bull...,"Prominent figures such as Deepak Parekh, Naray...",https://www.moneycontrol.com/news/business/out...,https://images.moneycontrol.com/static-mcnews/...,2023-12-29,"Ahead of the New Year 2024, India Inc bigwigs ...",None,Moneycontrol,0.0
98,The Hills Times,"HDFC Bank, IDA signs MoU to boost oral healthc...","HT Bureau GUWAHATI, Dec 13: HDFC Bank, India’s...",https://thehillstimes.in/business/hdfc-bank-id...,https://thehillstimes.in/wp-content/uploads/20...,2023-12-13,"HT Bureau\r\nGUWAHATI, Dec 13: HDFC Bank, Indi...",None,Thehillstimes.in,1.0


In [22]:
df_article.to_csv('output.csv', index=False)